# 1.
\begin{align*}
\min\ & 3x_2+ 4x_3 + 5x_4 \\
\text{t.q. } & x_1+x_3\geq 3 \\
& x_2+2x_3+x_4\geq 4 \\
& x_4 \geq 2 \\
& x_i\geq 0, i= 1,2,3,4
\end{align*}

### a) résoudre avec la méthode du simplexe duale

On va évidement effectuer nos calculs avec Julia. On code la fonction `pivot!(M, i, j)` qui fait un pivot sur la matrice `A` sur l'élément `i, j`. La matrice `st` contient toutes l'information nécéssaire à l'algorithme du simplex. La  dernière colonne contiendra les termes de droite et la dernière ligne contiendra les coûts réduits ainsi que la valeur de la fonction objective.

In [1]:
using LinearAlgebra, Suppressor

In [2]:
function pivot!(M, i, j)
    @assert M[i,j] != 0
    M[i, :] /= M[i,j]
    m, n = size(M)
    for k in setdiff(1:m, i)
        M[k,:] -=  M[k,j]*M[i,:]
    end
    M
end

pivot! (generic function with 1 method)

In [3]:
A = [1 0 1 0; 0 1  2 1; 0 0 0 1]
c = [0,3,4,5]
b = [3,4,2]
M = [A -I]

st = Rational[M b; c' zeros(Int, 3)' 0]

4×8 Array{Rational,2}:
 1//1  0//1  1//1  0//1  -1//1   0//1   0//1  3//1
 0//1  1//1  2//1  1//1   0//1  -1//1   0//1  4//1
 0//1  0//1  0//1  1//1   0//1   0//1  -1//1  2//1
 0//1  3//1  4//1  5//1   0//1   0//1   0//1  0//1

In [4]:
B_idx = [5,6,7]
println("Base = $B_idx")
pivot!(st, 1, 5)
pivot!(st, 2, 6)
pivot!(st, 3, 7)

Base = [5, 6, 7]


4×8 Array{Rational,2}:
 -1//1   0//1  -1//1   0//1  1//1  0//1  0//1  -3//1
  0//1  -1//1  -2//1  -1//1  0//1  1//1  0//1  -4//1
  0//1   0//1   0//1  -1//1  0//1  0//1  1//1  -2//1
  0//1   3//1   4//1   5//1  0//1  0//1  0//1   0//1

On décide de choisir (arbitrairement) comme variante sortante celle avec le terme de droite le plus négatif. la seconde variable de base va donc sortir. On va faire entrer $x_3$ dans la base.

In [5]:
B_idx[2] = 3
println("Base = $B_idx")
pivot!(st, 2, 3)

Base = [5, 3, 7]

4×8 Array{Rational,2}:
 -1//1  1//2  0//1   1//2  1//1  -1//2  0//1  -1//1
  0//1  1//2  1//1   1//2  0//1  -1//2  0//1   2//1
  0//1  0//1  0//1  -1//1  0//1   0//1  1//1  -2//1
  0//1  1//1  0//1   3//1  0//1   2//1  0//1  -8//1

In [6]:
B_idx[3] = 4
println("Base = $B_idx")
pivot!(st, 3, 4)

Base = [5, 3, 4]

4×8 Array{Rational,2}:
 -1//1  1//2  0//1  0//1  1//1  -1//2   1//2   -2//1
  0//1  1//2  1//1  0//1  0//1  -1//2   1//2    1//1
  0//1  0//1  0//1  1//1  0//1   0//1  -1//1    2//1
  0//1  1//1  0//1  0//1  0//1   2//1   3//1  -14//1

In [7]:
B_idx[1] = 1
println("Base = $B_idx")
pivot!(st, 1, 1)

Base = [1, 3, 4]

4×8 Array{Rational,2}:
 1//1  -1//2  0//1  0//1  -1//1   1//2  -1//2    2//1
 0//1   1//2  1//1  0//1   0//1  -1//2   1//2    1//1
 0//1   0//1  0//1  1//1   0//1   0//1  -1//1    2//1
 0//1   1//1  0//1  0//1   0//1   2//1   3//1  -14//1

In [8]:
xstar = zeros(Rational, 7)
xstar[B_idx] = st[1:3, end]
@show xstar
@show vstar = xstar[1:4]'*c;

xstar = Rational[2//1, 0//1, 1//1, 2//1, 0//1, 0//1, 0//1]
vstar = (xstar[1:4])' * c = 14//1


Il est si facile de vérifier nos résultats qu'il serait idiot de ne pas le faire!!!

In [9]:
using JuMP, Clp
m = Model(Clp.Optimizer)
@variable(m, x[1:4] >= 0)
@constraint(m, c1, x[1]+x[3]>=3)
@constraint(m, c2, x[2]+2x[3]+x[4]>=4)
@constraint(m, c3, x[4]>=2)
@objective(m, Min, 3x[2]+4x[3]+5x[4])
m

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1260
┌ Info: Precompiling Clp [e2554f3b-3117-50c0-817c-e040a3ddf72d]
└ @ Base loading.jl:1260


A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: c1, c2, c3, x

In [10]:
@suppress optimize!(m)
@show xstar2 = value.(x)
@show vstar2 = objective_value(m);

xstar2 = value.(x) = [2.0, 0.0, 1.0, 2.0]
vstar2 = objective_value(m) = 14.0


### Bref.

On a la même solution.

### b) Déterminer une solution optimale  du dual avec la théorie des écarts complémentaires.

In [11]:
xstar

7-element Array{Rational,1}:
 2//1
 0//1
 1//1
 2//1
 0//1
 0//1
 0//1

comme $x^*_1\neq 0$, $x^*_3\neq 0$, $x^*_4\neq 0$, $\Rightarrow \lambda^Ta_i = c_i, i = 1,2,4$

In [12]:
indexs = [1,3,4]
Mtmp = A[:,indexs]'
λs = Mtmp\c[indexs]

3-element Array{Float64,1}:
 0.0
 2.0
 3.0

Encore une fois,  il  est facile de vérifier nos  résultats. Le dual sera un problème de maximisation avec comme objectif

$$
\max_\lambda 3\lambda_1 + 4\lambda_2+2\lambda_3
$$

Notre solution à la valeur:

In [13]:
dot([3,4,2],[0, 2, 3]) 

14

ce qui est cohérent considérant la dualité forte!!!

On peut continuer et nous amuser, on sait que 

$$
r_D^T = c_D^T - \lambda^T D \Rightarrow \lambda^T D = c_D^T - r_D^T \Rightarrow D^T\lambda = c_D - r_D\Rightarrow \lambda = (D^T)^{-1}(c_D - r_D)
$$

In [14]:
D_idx = [2, 5, 6, 7] 
D = [A -I][:, D_idx]

3×4 Array{Int64,2}:
 0  -1   0   0
 1   0  -1   0
 0   0   0  -1

In [15]:
λs = (D')\([c; zeros(3)][D_idx] - st[end, D_idx])

3-element Array{Float64,1}:
 -0.0
  1.9999999999999996
  3.0

ou encore, en utilisant $\lambda^T = c_B^TB^{-1} \Rightarrow \lambda = (B^{-1})^Tc_B$

In [16]:
B_inv = -st[1:3, 5:7]

3×3 Array{Rational{Int64},2}:
 1//1  -1//2   1//2
 0//1   1//2  -1//2
 0//1   0//1   1//1

In [17]:
B_inv'*c[B_idx]

3-element Array{Rational{Int64},1}:
 0//1
 2//1
 3//1

Malheureusement, seule la première méthode est la bonne car il était clairement indiqué d'utiliser les écarts complémentaire.

### c) en modifiant la deuxième contrainte en 

$$
x_2 + 2x_3+x_4 \geq 4.1
$$


In [18]:
B_inv = -st[1:3, 5:7]

3×3 Array{Rational{Int64},2}:
 1//1  -1//2   1//2
 0//1   1//2  -1//2
 0//1   0//1   1//1

Le signe négatif vient du fait que la matrice initiale contient l'opposé de l'identité, non l'identité, en raison des variables de surplus.

In [19]:
Δb = [0, 1/10, 0]
Δz = λs'*Δb

0.19999999999999996

on se rappel que la solution optimale était 14

In [20]:
14+Δz

14.2

On peut aussi obtenir la réponse en regardent la variation de la solution optimale.

In [21]:
Δx = B_inv*Δb

3-element Array{Float64,1}:
 -0.05
  0.05
  0.0

La variation de la fonction objectif est alors

In [22]:
Δz = dot(c[B_idx], Δx)

0.2

ce qui est conforme à la réponse précédent.